In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import time
import json
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from urllib.parse import unquote_plus, quote_plus
pd.set_option('display.max_columns', None)

In [3]:
def get_date_range(start, ndays):
    """
    Date format: YYYY-MM-DD
    """
    end = datetime.strftime(datetime.strptime(start, "%Y-%m-%d") + timedelta(days=ndays), "%Y-%m-%d")
    res = f"t_1h:[{start}T01:00:00Z TO {end}T00:00:00Z]"
    return quote_plus(res)

In [4]:
base = "https://parisdata.opendatasoft.com"

libelle = "4_Septembre"

filter_nd_amont = "&refine.libelle_nd_amont=4_Septembre-Choiseul"

url2019 = f"{base}/api/records/1.0/search/?dataset=comptages-routiers-permanents&q={get_date_range('2019-01-01', 365)}&rows=8760&sort=-t_1h&facet=libelle&facet=t_1h&" \
          f"facet=etat_trafic&facet=libelle_nd_amont&facet=libelle_nd_aval&refine.libelle={libelle}{filter_nd_amont}"
url2020 = f"{base}/api/records/1.0/search/?dataset=comptages-routiers-permanents&q={get_date_range('2020-01-01', 365)}&rows=8760&sort=-t_1h&facet=libelle&facet=t_1h&" \
          f"facet=etat_trafic&facet=libelle_nd_amont&facet=libelle_nd_aval&refine.libelle={libelle}{filter_nd_amont}"
url2021 = f"{base}/api/records/1.0/search/?dataset=comptages-routiers-permanents&q={get_date_range('2021-01-01', 365)}&rows=8760&sort=-t_1h&facet=libelle&facet=t_1h&" \
          f"facet=etat_trafic&facet=libelle_nd_amont&facet=libelle_nd_aval&refine.libelle={libelle}{filter_nd_amont}"
urls = [url2019, url2020, url2021]

In [5]:
dfs = []
for url in urls:
    time.sleep(5)
    req = requests.get(url)
    if req.status_code != 200:
        print(f"Error for dates{unquote_plus(url.split('&q=')[-1].split('&')[0])}")
        continue
    content = json.loads(req.content)
    if len(content["records"]) == 0:
        print(f"No data for dates {unquote_plus(url.split('&q=')[-1].split('&')[0])}")
        continue
    dfs.append(pd.DataFrame(content["records"]))

No data for dates t_1h:[2019-01-01T01:00:00Z TO 2020-01-01T00:00:00Z]


In [6]:
df = pd.json_normalize(pd.concat(dfs)["fields"])
df.t_1h = df.t_1h.apply(lambda x: ":".join(x.split(":")[:2]).replace("T", " "))
df["day"] = df.t_1h.apply(lambda x: x.split(" ")[0])
df_ok = df[["t_1h", "day", "libelle", "etat_barre", "etat_trafic", "k", "q"]]

In [7]:
df_ok

,t_1h,day,libelle,etat_barre,etat_trafic,k,q
0,2020-11-01 03:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
1,2020-11-01 04:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
2,2020-11-01 05:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
3,2020-11-01 06:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
4,2020-11-01 07:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
...,...,...,...,...,...,...,...
9349,2021-12-06 19:00,2021-12-06,4_Septembre,Invalide,Fluide,5.59500,869.0
9350,2021-12-06 20:00,2021-12-06,4_Septembre,Invalide,Fluide,4.77334,716.0
9351,2021-12-06 21:00,2021-12-06,4_Septembre,Invalide,Fluide,3.02222,444.0
9352,2021-12-06 22:00,2021-12-06,4_Septembre,Invalide,Fluide,2.68778,364.0


In [8]:
df_ok[df_ok.k.isnull()]

,t_1h,day,libelle,etat_barre,etat_trafic,k,q
0,2020-11-01 03:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
1,2020-11-01 04:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
2,2020-11-01 05:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
3,2020-11-01 06:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
4,2020-11-01 07:00,2020-11-01,4_Septembre,Invalide,Inconnu,NaN,NaN
...,...,...,...,...,...,...,...
8941,2021-11-19 19:00,2021-11-19,4_Septembre,Invalide,Inconnu,NaN,NaN
9340,2021-12-06 10:00,2021-12-06,4_Septembre,Invalide,Inconnu,NaN,NaN
9341,2021-12-06 11:00,2021-12-06,4_Septembre,Invalide,Inconnu,NaN,NaN
9342,2021-12-06 12:00,2021-12-06,4_Septembre,Invalide,Inconnu,NaN,NaN


In [12]:
df_ok.etat_trafic.value_counts()

Inconnu       6659
Fluide        2686
Pré-saturé       6
Saturé           2
Bloqué           1
Name: etat_trafic, dtype: int64